# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from citipy import citipy

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
#Import cities csv as data frame
file_path= os.path.join('../', 'output_data','weather_data.csv') 
cities_df=pd.read_csv(file_path, index_col=0)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Iqaluit,63.7506,-68.5145,250.37,77,1,8.75,CA,1612052891
1,Pinega,64.7001,43.3936,262.20,97,93,2.26,RU,1612053016
2,Rikitea,-23.1203,-134.9692,299.44,73,16,5.01,PF,1612052916
3,Sisimiut,66.9395,-53.6735,258.87,90,74,3.42,GL,1612053016
4,Kavieng,-2.5744,150.7967,301.65,73,99,6.58,PG,1612052834
...,...,...,...,...,...,...,...,...,...
546,Kulhudhuffushi,6.6221,73.0700,300.74,65,23,2.62,MV,1612053108
547,Strezhevoy,60.7333,77.5889,244.15,69,40,8.00,RU,1612053108
548,Vilhena,-12.7406,-60.1458,295.15,100,0,2.06,BR,1612053108
549,Galeras,9.1610,-75.0481,299.10,65,19,4.61,CO,1612053109


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
locations = cities_df[["Lat", "Lng"]]

humidity=cities_df["Humidity"].astype(float)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig=gmaps.figure(layout=figure_layout)

heat_layer= gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=5)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
cloud_filter=cities_df.loc[:, 'Cloudiness']<7
cloud_ideal=cities_df.loc[cloud_filter]

wind_filter=cloud_ideal.loc[:, 'Wind Speed']<25
wind_ideal=cloud_ideal.loc[wind_filter]

temp_filter=(wind_ideal['Max Temp']>294)&(wind_ideal['Max Temp']<300)  #(cities_df['Max Temp']>340)&(cities_df['Max Temp']<350)
temp_ideal=wind_ideal[temp_filter]
temp_ideal["Hotel Name"]= " "
hotel_df=pd.DataFrame(temp_ideal)
hotel_df.head()

<ipython-input-26-64d02eb18b2d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_ideal["Hotel Name"]= " "


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
72,Natitingou,10.3042,1.3796,298.15,31,0,3.09,BJ,1612053028,
90,Saint-Philippe,-21.3585,55.7679,299.15,65,0,3.09,RE,1612053031,
93,Suleja,9.1758,7.1808,298.15,38,0,3.09,NG,1612052900,
106,Caravelas,-17.7125,-39.2481,297.24,85,0,2.27,BR,1612053033,
126,Beloha,-25.1667,45.0500,297.01,80,2,1.62,MG,1612052862,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"


lat_long=''
distance=5000
target_type='lodging'

query_parameters={
    'key': g_key,
    'location':lat_long,
    'radius':distance,
    'type':target_type
}

for index, each_row in hotel_df.iterrows():
    
    try:
        
        query_parameters['location']=f"{each_row['Latitude']}, {each_row['Longitude']}"
        response=requests.get(base_url,params=query_parameters)
        data=response.json()
        hotel_df.loc[index,"Hotel Name"]=data['results'][0]['name']
    
    except:
        print(f"Unable to locate a hotel in {each_row['City']} within {distance} meters.")


Unable to locate a hotel in Natitingou within 5000 meters.
Unable to locate a hotel in Saint-Philippe within 5000 meters.
Unable to locate a hotel in Suleja within 5000 meters.
Unable to locate a hotel in Caravelas within 5000 meters.
Unable to locate a hotel in Beloha within 5000 meters.
Unable to locate a hotel in Pisco within 5000 meters.
Unable to locate a hotel in Puerto Baquerizo Moreno within 5000 meters.
Unable to locate a hotel in São João da Barra within 5000 meters.
Unable to locate a hotel in Touho within 5000 meters.
Unable to locate a hotel in Warmbad within 5000 meters.
Unable to locate a hotel in Cabo Rojo within 5000 meters.
Unable to locate a hotel in Ouahigouya within 5000 meters.
Unable to locate a hotel in Saint-Pierre within 5000 meters.
Unable to locate a hotel in Ambilobe within 5000 meters.
Unable to locate a hotel in Iraquara within 5000 meters.
Unable to locate a hotel in León within 5000 meters.
Unable to locate a hotel in Kaeo within 5000 meters.
Unable to 

In [33]:
clean_hotel_df = hotel_df.dropna(how='any')

In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content =hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…